In [12]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 28.1 MB/s 
     |████████████████████████████████| 1.6 MB 67.0 MB/s 


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
tf.__version__

'2.8.2'

In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [6]:
(train_images, train_labels),(test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [7]:
train_images = train_images/255.0
test_images = test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters= hp.Int('conv_1_filter', min_value =32, max_value = 128, step=16),
          kernel_size = hp.Choice('conv_1_kernel', values =[3,5]),
          activation ='relu',
          input_shape =(28,28,1)
      ),
      keras.layers.Conv2D(
          filters= hp.Int('conv_2_filter', min_value =32, max_value = 34, step=16),
          kernel_size = hp.Choice('conv_2_kernel', values =[3,5]),
          activation ='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation = 'softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning rate', values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters 

In [15]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name='mnist Fashion')

In [21]:
tuner_search.search(train_images, train_labels, epochs=4,validation_split=0.1)

In [22]:
model = tuner_search.get_best_models(num_models=1)[0]

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        32288     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 64)                1179712   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,213,770
Trainable params: 1,213,770
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1,
          initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1324 - accuracy: 0.9511 - val_loss: 0.2615 - val_accuracy: 0.9122
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0904 - accuracy: 0.9666 - val_loss: 0.3065 - val_accuracy: 0.9155
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0636 - accuracy: 0.9766 - val_loss: 0.3247 - val_accuracy: 0.9117
Epoch 7/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0466 - accuracy: 0.9833 - val_loss: 0.4061 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0332 - accuracy: 0.9879 - val_loss: 0.4291 - val_accuracy: 0.9127
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0287 - accuracy: 0.9902 - val_loss: 0.4686 - val_accuracy: 0.9173
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0240 - accuracy: 0.9917 - val_loss: 0.5198 - val_accuracy